In [2]:
import sqlite3
import pandas as pd
import numpy as np 
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
pd.options.mode.chained_assignment = None  # default='warn'

pd.set_option('display.max_columns', None)
pd.set_option("max_rows", 30)

#!pip install thefuzz
#!pip install rapidfuzz
#!pip install python-Levenshtein

from thefuzz import process, fuzz

df_games = pd.read_csv("originales/games.csv")
df_teams = pd.read_csv("originales/teams.csv")
df_teamstats = pd.read_csv("originales/teamstats.csv")
df_leagues = pd.read_csv("originales/leagues.csv")
df_appearances = pd.read_csv("originales/appearances.csv")
df_players = pd.read_csv("originales/players.csv", encoding="ISO-8859-1")
df_shots = pd.read_csv("originales/shots.csv")



""" df_games = pd.read_csv("https://raw.githubusercontent.com/campeone-pixel/soccer_database/master/originales/games.csv")
df_teams = pd.read_csv("https://raw.githubusercontent.com/campeone-pixel/soccer_database/master/originales/teams.csv")
df_teamstats = pd.read_csv("https://raw.githubusercontent.com/campeone-pixel/soccer_database/master/originales/teamstats.csv")
df_leagues = pd.read_csv("https://raw.githubusercontent.com/campeone-pixel/soccer_database/master/originales/leagues.csv")
df_appearances = pd.read_csv("https://raw.githubusercontent.com/campeone-pixel/soccer_database/master/originales/appearances.csv")
df_players = pd.read_csv("https://raw.githubusercontent.com/campeone-pixel/soccer_database/master/originales/players.csv", encoding="ISO-8859-1")
df_shots = pd.read_csv("https://raw.githubusercontent.com/campeone-pixel/soccer_database/master/originales/shots.csv") """

' df_games = pd.read_csv("https://raw.githubusercontent.com/campeone-pixel/soccer_database/master/originales/games.csv")\ndf_teams = pd.read_csv("https://raw.githubusercontent.com/campeone-pixel/soccer_database/master/originales/teams.csv")\ndf_teamstats = pd.read_csv("https://raw.githubusercontent.com/campeone-pixel/soccer_database/master/originales/teamstats.csv")\ndf_leagues = pd.read_csv("https://raw.githubusercontent.com/campeone-pixel/soccer_database/master/originales/leagues.csv")\ndf_appearances = pd.read_csv("https://raw.githubusercontent.com/campeone-pixel/soccer_database/master/originales/appearances.csv")\ndf_players = pd.read_csv("https://raw.githubusercontent.com/campeone-pixel/soccer_database/master/originales/players.csv", encoding="ISO-8859-1")\ndf_shots = pd.read_csv("https://raw.githubusercontent.com/campeone-pixel/soccer_database/master/originales/shots.csv") '

In [3]:
df_teamstats_away = df_teamstats.loc[df_teamstats["location"] == "a"]
df_teamstats_home = df_teamstats.loc[df_teamstats["location"] == "h"]
df_teamstats_merged = pd.merge(
    df_teamstats_home, df_teamstats_away, on="gameID", suffixes=["_home", "_away"]
)

df_games_merged_with_teamstats = pd.merge(
    df_games, df_teamstats_merged, on="gameID", suffixes=("", "_remove")
)
df_games_merged_with_teamstats = df_games_merged_with_teamstats.drop(
    df_games_merged_with_teamstats.iloc[:, 13:34], axis=1
)
df_games_merged_with_teamstats = df_games_merged_with_teamstats.drop(
    ["homeProbability", "drawProbability", "awayProbability"], axis=1
)
df_games_merged_with_teamstats = df_games_merged_with_teamstats.drop(
    ["teamID_home", "teamID_away"], axis=1
)

df_games_merged_with_leagues = pd.merge(
    df_games_merged_with_teamstats, df_leagues, on="leagueID"
)

df_games_merged_with_team = pd.merge(
    df_games_merged_with_leagues,
    df_teams,
    left_on=["homeTeamID"],
    right_on=["teamID"],
    how="left",
)
df_games_merged_with_team = pd.merge(
    df_games_merged_with_team,
    df_teams,
    left_on=["awayTeamID"],
    right_on=["teamID"],
    how="left",
)

df_f_game = df_games_merged_with_team.rename(
    columns={"name_x": "liga", "name_y": "home_name", "name": "away_name"}
)
df_f_game = df_f_game.drop(
    [
        "understatNotation",
        "season_home",
        "date_home",
        "location_home",
        "goals_away",
        "goals_home",
        "season_away",
        "date_away",
        "location_away",
        "location_away",
        "teamID_x",
        "teamID_y",
    ],
    axis=1,
)


# df_appearances = pd.read_csv("appearances.csv")
# df_players = pd.read_csv("players.csv", encoding="ISO-8859-1")
# df_shots = pd.read_csv("shots.csv")

df_apperance_merged_with_players = pd.merge(
    df_appearances, df_players, on="playerID"
)





In [4]:
df2_appearances = pd.read_csv("players/appearances.csv")

df2_clubs = pd.read_csv("players/clubs.csv")
df2_competitions = pd.read_csv("players/competitions.csv")
df2_games = pd.read_csv("players/games.csv")
df2_player_val = pd.read_csv("players/player_valuations.csv")
df2_players = pd.read_csv("players/players.csv")


""" df2_appearances = pd.read_csv("https://raw.githubusercontent.com/campeone-pixel/soccer_database/master/players/appearances.csv")
df2_club_games = pd.read_csv("https://raw.githubusercontent.com/campeone-pixel/soccer_database/master/players/club_games.csv")
df2_clubs = pd.read_csv("https://raw.githubusercontent.com/campeone-pixel/soccer_database/master/players/clubs.csv")
df2_competitions = pd.read_csv("https://raw.githubusercontent.com/campeone-pixel/soccer_database/master/players/competitions.csv")
df2_games = pd.read_csv("https://raw.githubusercontent.com/campeone-pixel/soccer_database/master/players/games.csv")
df2_player_val= pd.read_csv("https://raw.githubusercontent.com/campeone-pixel/soccer_database/master/players/player_valuations.csv")
df2_players = pd.read_csv("https://raw.githubusercontent.com/campeone-pixel/soccer_database/master/players/players.csv") """


' df2_appearances = pd.read_csv("https://raw.githubusercontent.com/campeone-pixel/soccer_database/master/players/appearances.csv")\ndf2_club_games = pd.read_csv("https://raw.githubusercontent.com/campeone-pixel/soccer_database/master/players/club_games.csv")\ndf2_clubs = pd.read_csv("https://raw.githubusercontent.com/campeone-pixel/soccer_database/master/players/clubs.csv")\ndf2_competitions = pd.read_csv("https://raw.githubusercontent.com/campeone-pixel/soccer_database/master/players/competitions.csv")\ndf2_games = pd.read_csv("https://raw.githubusercontent.com/campeone-pixel/soccer_database/master/players/games.csv")\ndf2_player_val= pd.read_csv("https://raw.githubusercontent.com/campeone-pixel/soccer_database/master/players/player_valuations.csv")\ndf2_players = pd.read_csv("https://raw.githubusercontent.com/campeone-pixel/soccer_database/master/players/players.csv") '

In [5]:
top_5_ligas = df2_competitions.loc[
    (df2_competitions["pretty_name"] == "Bundesliga")
    | (df2_competitions["pretty_name"] == "Laliga")
    | (df2_competitions["pretty_name"] == "Premier League")
    | (df2_competitions["pretty_name"] == "Serie A")
    | (df2_competitions["pretty_name"] == "Ligue 1")
]


def match_ligue(x, col, score):
    match = process.extractOne(x, col, score_cutoff=score, scorer=fuzz.token_sort_ratio)
    if match:
        return match[2]
    else:
        return 2


# ---------------------------------------------------------------------------------------------------------
# top_5_ligas["pretty_name"]      df_teams.name
top_5_ligas["liga_new_id"] = df_leagues.loc[
    top_5_ligas["pretty_name"]
    .apply(lambda x: match_ligue(x, df_leagues.name, 50))
    .values,
    ["leagueID"],
].values
# ---------------------------------------------------------------------------------------------------------


df2_clubs_top5 = df2_clubs[
    df2_clubs["domestic_competition_id"].isin(top_5_ligas["competition_id"])
]


df2_games_top5 = df2_games[
    df2_games["competition_id"].isin(top_5_ligas["competition_id"])
]
seasons = df_f_game["season"].unique()


df2_games_top5 = df2_games_top5[df2_games_top5["season"].isin(seasons)]
df2_games_top5 = df2_games_top5[
    df2_games_top5["competition_id"].isin(top_5_ligas["competition_id"])
]


In [6]:
df2_games_top5 = pd.merge(df2_games_top5, top_5_ligas, on="competition_id")


df2_appearances_top5 = df2_appearances[
    df2_appearances["game_id"].isin(df2_games_top5["game_id"])
]
df2_players_top5 = df2_players[
    df2_players["current_club_id"].isin(df2_clubs_top5["club_id"])
]


# -----------------------------------------------------------------------------------------
# df2_players_top5['pretty_name']      df_players.name
# df2_players_top5['new_id'] = df_players.loc[df2_players_top5.apply(lambda x: match(x,df_players,70) ).values,"name"].values
# ----------------------------------------------------------------------------------------------


df2_players_val_top5 = df2_player_val[
    df2_player_val["player_id"].isin(df2_players_top5["player_id"])
]


df2_games_top5["date"] = pd.to_datetime(df2_games_top5["date"], format="%Y/%m/%d")
df_f_game["date"] = pd.to_datetime(df_f_game["date"], format="%Y/%m/%d")


df2_games_top5["month_year"] = df2_games_top5["date"].dt.to_period("M")
df_f_game["month_year"] = df_f_game["date"].dt.to_period("M")

df_f_game["match_name"] = df_f_game["home_name"] + " vs " + df_f_game["away_name"]

nan_values = pd.DataFrame(np.nan, index=[0], columns=df_f_game.columns)

df_f_game = df_f_game.append(nan_values, ignore_index=True)
df_f_game['date'] = pd.to_datetime(df_f_game['date']).dt.date


In [7]:
df2_games_top5.loc[
    df2_games_top5["club_home_pretty_name"] == "Fc Turin", "club_home_pretty_name"
] = "Torino"
df2_games_top5.loc[
    df2_games_top5["club_away_pretty_name"] == "Fc Turin", "club_away_pretty_name"
] = "Torino"

df2_games_top5.loc[
    df2_games_top5["club_away_pretty_name"].str.contains("Mailand"),
    "club_away_pretty_name",
] = "Ac Milan"
df2_games_top5.loc[
    df2_games_top5["club_home_pretty_name"].str.contains("Mailand"),
    "club_home_pretty_name",
] = "Ac Milan"

df2_games_top5.loc[
    df2_games_top5["club_away_pretty_name"].str.contains("Juventus"),
    "club_away_pretty_name",
] = "Juventus"
df2_games_top5.loc[
    df2_games_top5["club_home_pretty_name"].str.contains("Juventus"),
    "club_home_pretty_name",
] = "Juventus"

df2_games_top5.loc[
    df2_games_top5["club_away_pretty_name"].str.contains("Fc Granada"),
    "club_away_pretty_name",
] = "Granada"
df2_games_top5.loc[
    df2_games_top5["club_home_pretty_name"].str.contains("Fc Granada"),
    "club_home_pretty_name",
] = "Granada"

df2_games_top5.loc[
    df2_games_top5["club_away_pretty_name"] == "Real Sociedad San Sebastian",
    "club_away_pretty_name",
] = "Real Sociedad"
df2_games_top5.loc[
    df2_games_top5["club_home_pretty_name"] == "Real Sociedad San Sebastian",
    "club_home_pretty_name",
] = "Real Sociedad"

df2_games_top5.loc[
    df2_games_top5["club_away_pretty_name"] == "Stoke City", "club_away_pretty_name"
] = "Stoke"
df2_games_top5.loc[
    df2_games_top5["club_home_pretty_name"] == "Stoke City", "club_home_pretty_name"
] = "Stoke"

df2_games_top5.loc[
    df2_games_top5["club_away_pretty_name"] == "Swansea City", "club_away_pretty_name"
] = "Swansea"
df2_games_top5.loc[
    df2_games_top5["club_home_pretty_name"] == "Swansea City", "club_home_pretty_name"
] = "Swansea"

df2_games_top5.loc[
    df2_games_top5["club_home_pretty_name"] == "Ud Las Palmas", "club_home_pretty_name"
] = "Las Palmas"
df2_games_top5.loc[
    df2_games_top5["club_away_pretty_name"] == "Ud Las Palmas", "club_away_pretty_name"
] = "Las Palmas"

df2_games_top5.loc[
    df2_games_top5["club_home_pretty_name"] == "Ssc Neapel", "club_home_pretty_name"
] = "Napoli"
df2_games_top5.loc[
    df2_games_top5["club_away_pretty_name"] == "Ssc Neapel", "club_away_pretty_name"
] = "Napoli"

df2_games_top5.loc[
    df2_games_top5["club_home_pretty_name"] == "Hellas Verona", "club_home_pretty_name"
] = "Verona"
df2_games_top5.loc[
    df2_games_top5["club_away_pretty_name"] == "Hellas Verona", "club_away_pretty_name"
] = "Verona"

df2_games_top5.loc[
    df2_games_top5["club_home_pretty_name"] == "Us Lecce", "club_home_pretty_name"
] = "Lecce"
df2_games_top5.loc[
    df2_games_top5["club_away_pretty_name"] == "Us Lecce", "club_away_pretty_name"
] = "Lecce"

df2_games_top5.loc[
    df2_games_top5["club_home_pretty_name"] == "Atalanta Bergamo",
    "club_home_pretty_name",
] = "Atalanta"
df2_games_top5.loc[
    df2_games_top5["club_away_pretty_name"] == "Atalanta Bergamo",
    "club_away_pretty_name",
] = "Atalanta"

df2_games_top5.loc[
    df2_games_top5["club_home_pretty_name"] == "Brescia Calcio", "club_home_pretty_name"
] = "Brescia"
df2_games_top5.loc[
    df2_games_top5["club_away_pretty_name"] == "Brescia Calcio", "club_away_pretty_name"
] = "Brescia"

df2_games_top5.loc[
    df2_games_top5["club_home_pretty_name"] == "Ac Florenz", "club_home_pretty_name"
] = "Fiorentina"
df2_games_top5.loc[
    df2_games_top5["club_away_pretty_name"] == "Ac Florenz", "club_away_pretty_name"
] = "Fiorentina"


df2_games_top5.loc[
    df2_games_top5["club_home_pretty_name"] == "Spal", "club_home_pretty_name"
] = "SPAL 2013"
df2_games_top5.loc[
    df2_games_top5["club_away_pretty_name"] == "Spal", "club_away_pretty_name"
] = "SPAL 2013"


df2_games_top5.loc[
    df2_games_top5["club_home_pretty_name"] == "As Rom", "club_home_pretty_name"
] = "roma"
df2_games_top5.loc[
    df2_games_top5["club_away_pretty_name"] == "As Rom", "club_away_pretty_name"
] = "roma"

df2_games_top5.loc[
    df2_games_top5["club_home_pretty_name"] == "Us Sassuolo", "club_home_pretty_name"
] = "Sassuolo"
df2_games_top5.loc[
    df2_games_top5["club_away_pretty_name"] == "Us Sassuolo", "club_away_pretty_name"
] = "Sassuolo"

df2_games_top5.loc[
    df2_games_top5["club_home_pretty_name"] == "Lazio Rom", "club_home_pretty_name"
] = "Lazio"
df2_games_top5.loc[
    df2_games_top5["club_away_pretty_name"] == "Lazio Rom", "club_away_pretty_name"
] = "Lazio"

df2_games_top5.loc[
    df2_games_top5["club_home_pretty_name"] == "Genua Cfc", "club_home_pretty_name"
] = "genoa"
df2_games_top5.loc[
    df2_games_top5["club_away_pretty_name"] == "Genua Cfc", "club_away_pretty_name"
] = "genoa"

df2_games_top5.loc[
    df2_games_top5["club_home_pretty_name"] == "Cagliari Calcio",
    "club_home_pretty_name",
] = "Cagliari"
df2_games_top5.loc[
    df2_games_top5["club_away_pretty_name"] == "Cagliari Calcio",
    "club_away_pretty_name",
] = "Cagliari"

df2_games_top5.loc[
    df2_games_top5["club_home_pretty_name"] == "Sampdoria Genua",
    "club_home_pretty_name",
] = "Sampdoria"
df2_games_top5.loc[
    df2_games_top5["club_away_pretty_name"] == "Sampdoria Genua",
    "club_away_pretty_name",
] = "Sampdoria"

df2_games_top5.loc[
    df2_games_top5["club_home_pretty_name"] == "Delfino Pescara 1936",
    "club_home_pretty_name",
] = "Pescara"
df2_games_top5.loc[
    df2_games_top5["club_away_pretty_name"] == "Delfino Pescara 1936",
    "club_away_pretty_name",
] = "Pescara"


df2_games_top5.loc[
    df2_games_top5["club_home_pretty_name"] == "1 Fsv Mainz 05", "club_home_pretty_name"
] = "Mainz 05"
df2_games_top5.loc[
    df2_games_top5["club_away_pretty_name"] == "1 Fsv Mainz 05", "club_away_pretty_name"
] = "Mainz 05"

df2_games_top5.loc[
    df2_games_top5["club_home_pretty_name"] == "Fc Ingolstadt 04",
    "club_home_pretty_name",
] = "Ingolstadt"
df2_games_top5.loc[
    df2_games_top5["club_away_pretty_name"] == "Fc Ingolstadt 04",
    "club_away_pretty_name",
] = "Ingolstadt"

df2_games_top5.loc[
    df2_games_top5["club_home_pretty_name"] == "Sc Paderborn 07",
    "club_home_pretty_name",
] = "Paderborn"
df2_games_top5.loc[
    df2_games_top5["club_away_pretty_name"] == "Sc Paderborn 07",
    "club_away_pretty_name",
] = "Paderborn"

df2_games_top5.loc[
    df2_games_top5["club_home_pretty_name"] == "Sv Darmstadt 98",
    "club_home_pretty_name",
] = "Darmstadt"
df2_games_top5.loc[
    df2_games_top5["club_away_pretty_name"] == "Sv Darmstadt 98",
    "club_away_pretty_name",
] = "Darmstadt"

df2_games_top5.loc[
    df2_games_top5["club_home_pretty_name"] == "Fc Bayern Munchen",
    "club_home_pretty_name",
] = "Bayern Munich"
df2_games_top5.loc[
    df2_games_top5["club_away_pretty_name"] == "Fc Bayern Munchen",
    "club_away_pretty_name",
] = "Bayern Munich"

df2_games_top5.loc[
    df2_games_top5["club_home_pretty_name"] == "1 Fc Koln", "club_home_pretty_name"
] = "FC Cologne"
df2_games_top5.loc[
    df2_games_top5["club_away_pretty_name"] == "1 Fc Koln", "club_away_pretty_name"
] = "FC Cologne"

df2_games_top5.loc[
    df2_games_top5["club_home_pretty_name"] == "Hertha Bsc", "club_home_pretty_name"
] = "Hertha Berlin"
df2_games_top5.loc[
    df2_games_top5["club_away_pretty_name"] == "Hertha Bsc", "club_away_pretty_name"
] = "Hertha Berlin"

df2_games_top5.loc[
    df2_games_top5["club_home_pretty_name"] == "Tsg 1899 Hoffenheim",
    "club_home_pretty_name",
] = "Hoffenheim"
df2_games_top5.loc[
    df2_games_top5["club_away_pretty_name"] == "Tsg 1899 Hoffenheim",
    "club_away_pretty_name",
] = "Hoffenheim"

df2_games_top5.loc[
    df2_games_top5["club_home_pretty_name"] == "Hertha Bsc", "club_home_pretty_name"
] = "Hertha Berlin"
df2_games_top5.loc[
    df2_games_top5["club_away_pretty_name"] == "Hertha Bsc", "club_away_pretty_name"
] = "Hertha Berlin"

df2_games_top5.loc[
    df2_games_top5["club_home_pretty_name"] == "Hertha Bsc", "club_home_pretty_name"
] = "Hertha Berlin"
df2_games_top5.loc[
    df2_games_top5["club_away_pretty_name"] == "Hertha Bsc", "club_away_pretty_name"
] = "Hertha Berlin"


df2_games_top5.loc[
    df2_games_top5["club_home_pretty_name"] == "Palermo Fc", "club_home_pretty_name"
] = "Palermo"
df2_games_top5.loc[
    df2_games_top5["club_away_pretty_name"] == "Palermo Fc", "club_away_pretty_name"
] = "Palermo"

#-----------------------------------------------------------------------------------------------

df2_games_top5.loc[
    df2_games_top5["club_home_pretty_name"] == "Fc Girondins Bordeaux", "club_home_pretty_name"
] = "Bordeaux"
df2_games_top5.loc[
    df2_games_top5["club_away_pretty_name"] == "Fc Girondins Bordeaux", "club_away_pretty_name"
] = "Bordeaux"

df2_games_top5.loc[
    df2_games_top5["club_home_pretty_name"] == "Fc Stade Rennes", "club_home_pretty_name"
] = "Rennes"
df2_games_top5.loc[
    df2_games_top5["club_away_pretty_name"] == "Fc Stade Rennes", "club_away_pretty_name"
] = "Rennes"

df2_games_top5.loc[
    df2_games_top5["club_home_pretty_name"] == "Fc Lorient", "club_home_pretty_name"
] = "Lorient"
df2_games_top5.loc[
    df2_games_top5["club_away_pretty_name"] == "Fc Lorient", "club_away_pretty_name"
] = "Lorient"

df2_games_top5.loc[
    df2_games_top5["club_home_pretty_name"] == "Losc Lille", "club_home_pretty_name"
] = "Lille"
df2_games_top5.loc[
    df2_games_top5["club_away_pretty_name"] == "Losc Lille", "club_away_pretty_name"
] = "Lille"

df2_games_top5.loc[
    df2_games_top5["club_home_pretty_name"] == "Ogc Nizza", "club_home_pretty_name"
] = "Nizza"
df2_games_top5.loc[
    df2_games_top5["club_away_pretty_name"] == "Ogc Nizza", "club_away_pretty_name"
] = "Nizza"

df2_games_top5.loc[
    df2_games_top5["club_home_pretty_name"] == "Thonon Evian Grand Geneve Fc", "club_home_pretty_name"
] = "Evian Thonon Gaillard"
df2_games_top5.loc[
    df2_games_top5["club_away_pretty_name"] == "Thonon Evian Grand Geneve Fc", "club_away_pretty_name"
] = "Evian Thonon Gaillard"

df2_games_top5.loc[
    df2_games_top5["club_home_pretty_name"] == "Stade Reims", "club_home_pretty_name"
] = "Reims"
df2_games_top5.loc[
    df2_games_top5["club_away_pretty_name"] == "Stade Reims", "club_away_pretty_name"
] = "Reims"

df2_games_top5.loc[
    df2_games_top5["club_home_pretty_name"] == "Ea Guingamp", "club_home_pretty_name"
] = "Guingamp"
df2_games_top5.loc[
    df2_games_top5["club_away_pretty_name"] == "Ea Guingamp", "club_away_pretty_name"
] = "Guingamp"

df2_games_top5["match_name"] = (
    df2_games_top5["club_home_pretty_name"]
    + " vs "
    + df2_games_top5["club_away_pretty_name"]
)


In [8]:
def match_games(x, y, score):

    liga = x["liga_new_id"]
    date = x["date"]
   

    y = y.loc[
        (y["leagueID"] == liga)
        & (y["date"] == date)
    
    ]

    match = process.extractOne(
        x["match_name"], y["match_name"], score_cutoff=score, scorer=fuzz.WRatio
    )

    if match:
        return match[2]
    else:
        return 12680


# ---------------------------------------------------------------------------------------------------------

df2_games_top5[["match_name_df", "homeTeamID", "awayTeamID",'home_name']] = df_f_game.loc[
    df2_games_top5.apply(lambda x: match_games(x, df_f_game, 85), axis=1).values,
    ["match_name", "homeTeamID", "awayTeamID",'home_name'],
].values
# ---------------------------------------------------------------------------------------------------------


c:\Users\Mati\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\core\ops\array_ops.py:73: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  result = libops.scalar_compare(x.ravel(), y, op)


In [9]:
club_agrupados_home = df2_games_top5[
    ["home_club_id", "club_home_pretty_name", "homeTeamID"]
]

gb_club_agrupados_home = club_agrupados_home.groupby(["home_club_id", "homeTeamID"])[
    "club_home_pretty_name"
].size()
gb_club_agrupados_home
gb_club_agrupados_home = (
    gb_club_agrupados_home.reset_index(level="home_club_id")
    .groupby("home_club_id")["club_home_pretty_name"]
    .idxmax()
    .reset_index(name="homeTeamID")
)


df2_clubs_top5_only_names = df2_clubs_top5[["club_id", "pretty_name"]]


gb_club_agrupados_home = pd.merge(
    gb_club_agrupados_home,
    df2_clubs_top5_only_names,
    left_on="home_club_id",
    right_on="club_id",
)

gb_club_agrupados_home
gb_club_agrupados_home = pd.merge(
    gb_club_agrupados_home, df_teams, left_on="homeTeamID", right_on="teamID"
)
gb_club_agrupados_home = gb_club_agrupados_home.drop(
    columns=["home_club_id", "homeTeamID", "pretty_name", "name"]
)



In [10]:
df2_games_top5_game_id=df2_games_top5[['game_id','date','month_year','home_club_id','away_club_id']]
df_f_game_game_id=df_f_game[['gameID','date','month_year','homeTeamID','awayTeamID']]
df_f_game_game_id.drop(index=12680,inplace=True)


df2_games_top5_game_id=pd.merge(df2_games_top5_game_id,gb_club_agrupados_home,left_on='home_club_id',right_on='club_id')

df2_games_top5_game_id=pd.merge(df2_games_top5_game_id,gb_club_agrupados_home,left_on='away_club_id',right_on='club_id',suffixes=('_home','_away'))
df2_games_top5_game_id.drop(columns=['club_id_home','club_id_away'],inplace=True)
df2_games_top5_game_id.rename(columns={'teamID_home':"homeTeamID",'teamID_away':'awayTeamID'},inplace=True)

df_f_game_game_id['date']=pd.to_datetime(df_f_game_game_id.date)
df_f_game_game_id=df_f_game_game_id.sort_values(by='date')
df2_games_top5_game_id=df2_games_top5_game_id.sort_values(by='date')
df_f_game_game_id['homeTeamID']=df_f_game_game_id.homeTeamID.astype('int64')
df_f_game_game_id['awayTeamID']=df_f_game_game_id.awayTeamID.astype('int64')


df2_games_top5_game_id=pd.merge_asof(df2_games_top5_game_id,df_f_game_game_id,on='date', by=['homeTeamID','awayTeamID'], direction='nearest')


df2_games_top5_game_id.drop(columns=['date','month_year_x','month_year_y','home_club_id','away_club_id','awayTeamID','homeTeamID'],inplace=True)


df2_games_top5_imp=pd.merge(df2_games_top5,df2_games_top5_game_id,on='game_id')
df2_games_top5.drop(columns=['home_name'],inplace=True)


df2_games_top5_imp=df2_games_top5_imp[['gameID','round','season','url_x','url_y','attendance','home_club_id','away_club_id','game_id','competition_id','date']]

df_f_game=pd.merge(df_f_game,df2_games_top5_imp,on='gameID')

df_f_game.drop(columns=['season_x','month_year','match_name','date_x'],inplace=True)
df_f_game.rename(columns={'season_y':'season','date_y':'date'},inplace=True)
df2_appearances=df2_appearances[
    df2_appearances["game_id"].isin(df_f_game["game_id"])
]
df2_appearances


,appearance_id,game_id,player_id,player_club_id,date,player_pretty_name,competition_id,yellow_cards,red_cards,goals,assists,minutes_played
34,2486513_37787,2486513,37787,2288,2014-09-20,Marvin Emnes,GB1,0,0,0,0,5
35,2460649_119557,2460649,119557,42,2014-09-14,Kenan Karaman,L1,0,0,0,0,19
64,2460300_22388,2460300,22388,2288,2014-08-16,Bafetimbi Gomis,GB1,0,0,0,0,13
75,2486677_72768,2486677,72768,2288,2015-01-10,Nelson Oliveira,GB1,0,0,0,0,16
76,2486686_72768,2486686,72768,2288,2015-01-17,Nelson Oliveira,GB1,0,0,0,0,66
...,...,...,...,...,...,...,...,...,...,...,...,...
905941,3433343_167105,3433343,167105,252,2021-01-06,Lukas Lerager,IT1,0,0,0,0,90
905942,3433347_167105,3433347,167105,252,2021-01-09,Lukas Lerager,IT1,0,0,0,0,16
905943,3433355_167105,3433355,167105,252,2021-01-17,Lukas Lerager,IT1,0,0,0,0,45
907464,3394631_709726,3394631,709726,1421,2020-10-17,Hugo Ekitike,FR1,1,0,0,0,55


In [11]:
df_appearances

,gameID,playerID,goals,ownGoals,shots,xGoals,xGoalsChain,xGoalsBuildup,assists,keyPasses,xAssists,position,positionOrder,yellowCard,redCard,time,substituteIn,substituteOut,leagueID
0,81,560,0,0,0,0.000000,0.000000,0.000000,0,0,0.000000,GK,1,0,0,90,0,0,1
1,81,557,0,0,0,0.000000,0.106513,0.106513,0,1,0.106513,DR,2,0,0,82,222605,0,1
2,81,548,0,0,0,0.000000,0.127738,0.127738,0,0,0.000000,DC,3,0,0,90,0,0,1
3,81,628,0,0,0,0.000000,0.106513,0.106513,0,0,0.000000,DC,3,0,0,90,0,0,1
4,81,1006,0,0,0,0.000000,0.021225,0.021225,0,0,0.000000,DL,4,0,0,90,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
356508,16135,3509,0,0,1,0.028713,0.058495,0.058495,0,0,0.000000,MC,9,0,0,90,0,0,5
356509,16135,4882,0,0,1,0.013482,0.058495,0.045374,0,2,0.058495,MC,9,0,0,90,0,0,5
356510,16135,5786,0,0,0,0.000000,0.045374,0.045374,0,0,0.000000,FW,15,0,0,90,0,0,5
356511,16135,8997,0,0,0,0.000000,0.000000,0.000000,0,0,0.000000,FW,15,0,0,78,474075,0,5


In [12]:
df_f_game_game_id_date=df_f_game[['gameID','game_id','date']]
df_appearances=pd.merge(df_appearances,df_f_game_game_id_date)


In [13]:
df_appearances=pd.merge(df_appearances,df_players)
df_appearances

,gameID,playerID,goals,ownGoals,shots,xGoals,xGoalsChain,xGoalsBuildup,assists,keyPasses,xAssists,position,positionOrder,yellowCard,redCard,time,substituteIn,substituteOut,leagueID,game_id,date,name
0,81,560,0,0,0,0.000000,0.000000,0.000000,0,0,0.000000,GK,1,0,0,90,0,0,1,2577860,2015-08-08,Sergio Romero
1,91,560,0,0,0,0.000000,0.058938,0.058938,0,0,0.000000,GK,1,0,0,90,0,0,1,2577885,2015-08-14,Sergio Romero
2,101,560,0,0,0,0.000000,0.014860,0.014860,0,0,0.000000,GK,1,0,0,90,0,0,1,2577899,2015-08-22,Sergio Romero
3,120,560,0,0,0,0.000000,0.570477,0.570477,0,0,0.000000,GK,1,0,0,90,0,0,1,2577944,2015-08-30,Sergio Romero
4,3412,560,0,0,0,0.000000,0.054445,0.054445,0,0,0.000000,GK,1,0,0,90,0,0,1,2698328,2017-05-17,Sergio Romero
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
356536,16132,7396,0,0,0,0.000000,0.251697,0.251697,0,0,0.000000,DC,3,0,0,90,0,0,5,3394999,2021-05-23,Loic Bessile
356537,16134,9566,0,0,1,0.019131,0.019131,0.000000,0,0,0.000000,Sub,17,0,0,29,0,474027,5,3395001,2021-05-23,Yanis Lhéry
356538,16134,9565,0,0,0,0.000000,0.000000,0.000000,0,0,0.000000,Sub,17,0,0,12,0,474032,5,3395001,2021-05-23,Mathys Saban
356539,16134,9568,0,0,0,0.000000,0.000000,0.000000,0,0,0.000000,Sub,17,0,0,1,0,474045,5,3395001,2021-05-23,Charles Costes


In [14]:
df_appearances.reset_index()

df2_appearances.reset_index()
df2_appearances

,appearance_id,game_id,player_id,player_club_id,date,player_pretty_name,competition_id,yellow_cards,red_cards,goals,assists,minutes_played
34,2486513_37787,2486513,37787,2288,2014-09-20,Marvin Emnes,GB1,0,0,0,0,5
35,2460649_119557,2460649,119557,42,2014-09-14,Kenan Karaman,L1,0,0,0,0,19
64,2460300_22388,2460300,22388,2288,2014-08-16,Bafetimbi Gomis,GB1,0,0,0,0,13
75,2486677_72768,2486677,72768,2288,2015-01-10,Nelson Oliveira,GB1,0,0,0,0,16
76,2486686_72768,2486686,72768,2288,2015-01-17,Nelson Oliveira,GB1,0,0,0,0,66
...,...,...,...,...,...,...,...,...,...,...,...,...
905941,3433343_167105,3433343,167105,252,2021-01-06,Lukas Lerager,IT1,0,0,0,0,90
905942,3433347_167105,3433347,167105,252,2021-01-09,Lukas Lerager,IT1,0,0,0,0,16
905943,3433355_167105,3433355,167105,252,2021-01-17,Lukas Lerager,IT1,0,0,0,0,45
907464,3394631_709726,3394631,709726,1421,2020-10-17,Hugo Ekitike,FR1,1,0,0,0,55


In [15]:
def match_games(x, y, score):

    game = x["game_id"]
    yellowCard = x["yellowCard"]
    redCard = x["redCard"]
    goals = x["goals"]
    assists = x["assists"]

   

    y = y.loc[
        (y["game_id"] == game)
        & (y["yellow_cards"] == yellowCard)
        & (y["red_cards"] == redCard)
        & (y["goals"] == goals)
        & (y["assists"] == assists)
    
    ]

    match = process.extractOne(
        x["name"], y["player_pretty_name"], score_cutoff=score, scorer=fuzz.WRatio
    )

    if match:
        return match[2]
    else:
        return 34


# ---------------------------------------------------------------------------------------------------------

df_appearances[['playerID_df2', "player_club_id", "appearance_id",'player_pretty_name']] = df2_appearances.loc[
    df_appearances.apply(lambda x: match_games(x, df2_appearances, 85), axis=1).values,
    ['player_id', "player_club_id", "appearance_id",'player_pretty_name'],
].values
# ---------------------------------------------------------------------------------------------------------


In [17]:
df_appearances


,gameID,playerID,goals,ownGoals,shots,xGoals,xGoalsChain,xGoalsBuildup,assists,keyPasses,xAssists,position,positionOrder,yellowCard,redCard,time,substituteIn,substituteOut,leagueID,game_id,date,name,playerID_df2,player_club_id,appearance_id,player_pretty_name
0,81,560,0,0,0,0.000000,0.000000,0.000000,0,0,0.000000,GK,1,0,0,90,0,0,1,2577860,2015-08-08,Sergio Romero,30690,985,2577860_30690,Sergio Romero
1,91,560,0,0,0,0.000000,0.058938,0.058938,0,0,0.000000,GK,1,0,0,90,0,0,1,2577885,2015-08-14,Sergio Romero,30690,985,2577885_30690,Sergio Romero
2,101,560,0,0,0,0.000000,0.014860,0.014860,0,0,0.000000,GK,1,0,0,90,0,0,1,2577899,2015-08-22,Sergio Romero,30690,985,2577899_30690,Sergio Romero
3,120,560,0,0,0,0.000000,0.570477,0.570477,0,0,0.000000,GK,1,0,0,90,0,0,1,2577944,2015-08-30,Sergio Romero,30690,985,2577944_30690,Sergio Romero
4,3412,560,0,0,0,0.000000,0.054445,0.054445,0,0,0.000000,GK,1,0,0,90,0,0,1,2698328,2017-05-17,Sergio Romero,30690,985,2698328_30690,Sergio Romero
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
356536,16132,7396,0,0,0,0.000000,0.251697,0.251697,0,0,0.000000,DC,3,0,0,90,0,0,5,3394999,2021-05-23,Loic Bessile,371150,40,3394999_371150,Loic Bessile
356537,16134,9566,0,0,1,0.019131,0.019131,0.000000,0,0,0.000000,Sub,17,0,0,29,0,474027,5,3395001,2021-05-23,Yanis Lhéry,654414,618,3395001_654414,Yanis Lhery
356538,16134,9565,0,0,0,0.000000,0.000000,0.000000,0,0,0.000000,Sub,17,0,0,12,0,474032,5,3395001,2021-05-23,Mathys Saban,783799,618,3395001_783799,Mathys Saban
356539,16134,9568,0,0,0,0.000000,0.000000,0.000000,0,0,0.000000,Sub,17,0,0,1,0,474045,5,3395001,2021-05-23,Charles Costes,854973,2969,3395001_854973,Charles Elyan Costes
